機械学習用の特徴量

In [324]:
import pandas as pd
import pandas.tseries.offsets as offsets
import os
from sklearn.preprocessing import OneHotEncoder
import pandas_profiling as pdp
from sklearn.preprocessing import LabelEncoder
import japandas as jpd
import jpholiday
import datetime
from IPython.display import display
import numpy as np


In [325]:
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 100)

In [345]:
train = pd.read_csv("../data/train.csv")
pdp.ProfileReport(train)


Number of variables,13
Number of observations,3366
Total Missing (%),0.0%
Total size in memory,341.9 KiB
Average record size in memory,104.0 B
Numeric,4
Categorical,9
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [346]:
ex_train = pd.read_csv("../data/ex_total.csv")
ex_train["match_date"] = pd.to_datetime(ex_train["match_date"])
ex_train = ex_train.loc[ex_train["match_date"] < pd.to_datetime("2006-3-3"),:]
print(ex_train.shape)
pdp.ProfileReport(ex_train)


(5092, 14)


Number of variables,15
Number of observations,5092
Total Missing (%),2.8%
Total size in memory,596.8 KiB
Average record size in memory,120.0 B
Numeric,4
Categorical,8
Boolean,0
Date,1
Text (Unique),1
Rejected,1


In [347]:
#J1のデータだけ使う
ex_train = ex_train.loc[ex_train["division"] == 1,:]
print(ex_train.shape)

(3372, 14)


In [348]:
test = pd.read_csv("../data/test.csv")
pdp.ProfileReport(test)

Number of variables,12
Number of observations,477
Total Missing (%),0.9%
Total size in memory,44.8 KiB
Average record size in memory,96.2 B
Numeric,3
Categorical,9
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [349]:
total_df = pd.concat((ex_train,train))
total_df = pd.concat((total,test))
#total_df = pd.concat((train,test))
total_df = total.reset_index(drop=True)
print(total_df.shape)
total_df.head()

(7215, 14)


,attendance,away_team,broadcasters,division,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather
0,59626.0,横浜Ｍ,NaN,1.0,Ｖ川崎,NaN,49,19:29,1993-05-15 00:00:00,第１日,第１節,NaN,国立霞ヶ丘競技場,晴
1,14126.0,清水,NaN,1.0,横浜Ｆ,NaN,50,13:05,1993-05-16 00:00:00,第２日,第１節,NaN,横浜市三ツ沢総合公園球技場,晴
2,11875.0,市原,NaN,1.0,広島,NaN,51,13:59,1993-05-16 00:00:00,第２日,第１節,NaN,広島スタジアム,晴
3,10898.0,名古屋,NaN,1.0,鹿島,NaN,52,16:00,1993-05-16 00:00:00,第２日,第１節,NaN,県立カシマサッカースタジアム,晴
4,19580.0,浦和,NaN,1.0,Ｇ大阪,NaN,53,19:04,1993-05-16 00:00:00,第２日,第１節,NaN,万博記念競技場,晴


# チーム名表記ゆれ統一

In [350]:
total_df["away_team"] = total["away_team"].str.h2z()
total_df["home_team"] = total["home_team"].str.h2z()
total_df["venue"] = total["venue"].str.h2z()
pdp.ProfileReport(total)


Number of variables,14
Number of observations,7215
Total Missing (%),6.3%
Total size in memory,789.2 KiB
Average record size in memory,112.0 B
Numeric,3
Categorical,8
Boolean,1
Date,0
Text (Unique),1
Rejected,0


# スタジアムデータ結合

In [351]:
stadium = pd.read_csv("../data/stadium_capacity_mapping.csv")
ex_stadium = pd.read_csv("../data/ex_stadium_capacity_mapping_padding.csv")


In [352]:
s1 = set(stadium["stadium"])
s2 = set(ex_stadium["stadium"])
dupe = s1.intersection(s2)
dupe_index = []
for key,row in  ex_stadium.iterrows():
    if dupe.issuperset(set([row["stadium"]])):
        dupe_index.append(key)
ex_stadium = ex_stadium.drop(dupe_index)
total_stadium = pd.concat((stadium, ex_stadium))
total_stadium = total_stadium.reset_index(drop=True)
total_stadium.to_csv("../data/total_stadium.csv", index=False)

# match_report結合

In [353]:
#match_reports = pd.read_csv("../data/match_reports.csv")
#ex_match_reports = pd.read_csv("../data/ex_match_reports.csv")

In [354]:
#print(match_reports.shape)
#print(ex_match_reports.shape)

(3825, 25)
(14362, 25)


In [357]:
#total_match_reports = pd.concat((match_reports,ex_match_reports))
#total_match_reports = total_match_reports.drop_duplicates()
#total_match_reports.to_csv("../data/total_match_reports.csv", index=False)

# 得点と勝敗

In [358]:
# 選手名とかポジションとかよくわからないので
# 点数と勝敗だけいれる
#match_df = pd.read_csv("../data/total_match_reports.csv")
#match_df["home_team_win"] = match_df["home_team_score"] > match_df["away_team_score"]
#total_df = pd.merge(total_df,match_df[["id", "home_team_win", "home_team_score", "away_team_score"]],on="id",how="left")

In [359]:
#print(total_df.shape)
#total_df.head()

(7338, 17)


,attendance,away_team,broadcasters,division,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,home_team_win,home_team_score,away_team_score
0,59626.0,横浜Ｍ,NaN,1.0,Ｖ川崎,NaN,49,19:29,1993-05-15 00:00:00,第１日,第１節,NaN,国立霞ヶ丘競技場,晴,NaN,NaN,NaN
1,14126.0,清水,NaN,1.0,横浜Ｆ,NaN,50,13:05,1993-05-16 00:00:00,第２日,第１節,NaN,横浜市三ツ沢総合公園球技場,晴,NaN,NaN,NaN
2,11875.0,市原,NaN,1.0,広島,NaN,51,13:59,1993-05-16 00:00:00,第２日,第１節,NaN,広島スタジアム,晴,NaN,NaN,NaN
3,10898.0,名古屋,NaN,1.0,鹿島,NaN,52,16:00,1993-05-16 00:00:00,第２日,第１節,NaN,県立カシマサッカースタジアム,晴,NaN,NaN,NaN
4,19580.0,浦和,NaN,1.0,Ｇ大阪,NaN,53,19:04,1993-05-16 00:00:00,第２日,第１節,NaN,万博記念競技場,晴,NaN,NaN,NaN


# カレンダー

In [281]:
# datetime型に変換
total_df.loc[:, "match_date"] = pd.to_datetime(total_df["match_date"])
total_df.loc[:, "kick_off_time"] = pd.to_datetime(total_df["kick_off_time"])

total_df["match_date_year"] = total_df["match_date"].dt.year
total_df["match_date_month"] = total_df["match_date"].dt.month
total_df["match_date_day"] = total_df["match_date"].dt.day
total_df["match_date_dayofweek"] = total_df["match_date"].dt.dayofweek  # Monday=0, Sunday=6

In [282]:
#祝日判定
date = pd.date_range(start='1993-1-1', end='2018-12-31', freq='D')
date = pd.Series(date)
public_holiday = []
for i, d in date.iteritems():
    public_holiday.append(jpholiday.is_holiday(datetime.date(d.to_datetime().year,d.to_datetime().month,d.to_datetime().day)))

date_df = pd.DataFrame(date)
date_df.columns = ["date"]
p_holiday = pd.DataFrame(public_holiday)
p_holiday.columns = ["public_holiday"]

p_holiday_df = pd.concat((date_df,p_holiday),axis=1)
p_holiday_df["public_holiday"] = p_holiday_df["public_holiday"].astype(int)

total_df = pd.merge(total_df,p_holiday_df,left_on="match_date",right_on="date")
print(total_df.shape)
total_df.head()

/home/watanabe/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: to_datetime is deprecated. Use self.to_pydatetime()
  exec(code_obj, self.user_global_ns, self.user_ns)


(11040, 23)


,attendance,away_team,broadcasters,division,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,home_team_win,home_team_score,away_team_score,match_date_year,match_date_month,match_date_day,match_date_dayofweek,date,public_holiday
0,59626.0,横浜Ｍ,NaN,1.0,Ｖ川崎,NaN,49,2018-11-08 19:29:00,1993-05-15,第１日,第１節,NaN,国立霞ヶ丘競技場,晴,NaN,NaN,NaN,1993,5,15,5,1993-05-15,0
1,14126.0,清水,NaN,1.0,横浜Ｆ,NaN,50,2018-11-08 13:05:00,1993-05-16,第２日,第１節,NaN,横浜市三ツ沢総合公園球技場,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0
2,11875.0,市原,NaN,1.0,広島,NaN,51,2018-11-08 13:59:00,1993-05-16,第２日,第１節,NaN,広島スタジアム,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0
3,10898.0,名古屋,NaN,1.0,鹿島,NaN,52,2018-11-08 16:00:00,1993-05-16,第２日,第１節,NaN,県立カシマサッカースタジアム,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0
4,19580.0,浦和,NaN,1.0,Ｇ大阪,NaN,53,2018-11-08 19:04:00,1993-05-16,第２日,第１節,NaN,万博記念競技場,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0


In [283]:
total_df.loc[:, "section"] = total_df.loc[:, "section"].apply(lambda x: int(x[1:-1]))
total_df.loc[:, "round"] = total_df.loc[:, "round"].apply(lambda x: int(x[1:-1]))
# 2015年と2016年は2シーズン制
total_df.query("(match_date_year in [2015, 2016])&(section == 1)").iloc[7:12, :]

,attendance,away_team,broadcasters,division,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,home_team_win,home_team_score,away_team_score,match_date_year,match_date_month,match_date_day,match_date_dayofweek,date,public_holiday
8887,14671.0,甲府,スカパー!/スカパー!プレミアムサービス/NHK広島/NHK甲府,NaN,広島,85.0,16806,2018-11-08 14:04:00,2015-03-07,1,1,8.9,エディオンスタジアム広島,曇,True,2.0,0.0,2015,3,7,5,2015-03-07,0
8888,11580.0,新潟,スカパー!/スカパー!プレミアムサービス/NHK佐賀/NHK新潟,NaN,鳥栖,34.0,16807,2018-11-08 14:05:00,2015-03-07,1,1,14.5,ベストアメニティスタジアム,曇,True,2.0,1.0,2015,3,7,5,2015-03-07,0
8889,11580.0,新潟,スカパー!/スカパー!プレミアムサービス/NHK佐賀/NHK新潟,NaN,鳥栖,34.0,16807,2018-11-08 14:05:00,2015-03-07,1,1,14.5,ベストアメニティスタジアム,曇,True,2.0,1.0,2015,3,7,5,2015-03-07,0
8890,38123.0,川崎Ｆ,スカパー!/スカパー!プレミアムサービス/BS-TBS,NaN,横浜ＦＭ,61.0,16808,2018-11-08 15:03:00,2015-03-07,1,1,8.8,日産スタジアム,曇,False,1.0,3.0,2015,3,7,5,2015-03-07,0
8891,38123.0,川崎Ｆ,スカパー!/スカパー!プレミアムサービス/BS-TBS,NaN,横浜ＦＭ,61.0,16808,2018-11-08 15:03:00,2015-03-07,1,1,8.8,日産スタジアム,曇,False,1.0,3.0,2015,3,7,5,2015-03-07,0


In [284]:
# 両年とも1シーズンに直した時の18節は7月からなので、7月からのsectionを+17する
sec_index = total_df.query("(match_date_year in [2015, 2016])&(match_date_month >= 7)").index
total_df.loc[sec_index, "section"] = total_df.loc[sec_index, "section"] + 17


In [285]:
total_df.head()

,attendance,away_team,broadcasters,division,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,home_team_win,home_team_score,away_team_score,match_date_year,match_date_month,match_date_day,match_date_dayofweek,date,public_holiday
0,59626.0,横浜Ｍ,NaN,1.0,Ｖ川崎,NaN,49,2018-11-08 19:29:00,1993-05-15,1,1,NaN,国立霞ヶ丘競技場,晴,NaN,NaN,NaN,1993,5,15,5,1993-05-15,0
1,14126.0,清水,NaN,1.0,横浜Ｆ,NaN,50,2018-11-08 13:05:00,1993-05-16,2,1,NaN,横浜市三ツ沢総合公園球技場,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0
2,11875.0,市原,NaN,1.0,広島,NaN,51,2018-11-08 13:59:00,1993-05-16,2,1,NaN,広島スタジアム,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0
3,10898.0,名古屋,NaN,1.0,鹿島,NaN,52,2018-11-08 16:00:00,1993-05-16,2,1,NaN,県立カシマサッカースタジアム,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0
4,19580.0,浦和,NaN,1.0,Ｇ大阪,NaN,53,2018-11-08 19:04:00,1993-05-16,2,1,NaN,万博記念競技場,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0


In [286]:
total_df = total_df.merge(right=total_stadium,
                          how="left",
                          left_on="venue",
                          right_on="stadium").drop("stadium", axis=1)

In [287]:
total_df.head()

,attendance,away_team,broadcasters,division,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,home_team_win,home_team_score,away_team_score,match_date_year,match_date_month,match_date_day,match_date_dayofweek,date,public_holiday,capacity
0,59626.0,横浜Ｍ,NaN,1.0,Ｖ川崎,NaN,49,2018-11-08 19:29:00,1993-05-15,1,1,NaN,国立霞ヶ丘競技場,晴,NaN,NaN,NaN,1993,5,15,5,1993-05-15,0,48000.0
1,14126.0,清水,NaN,1.0,横浜Ｆ,NaN,50,2018-11-08 13:05:00,1993-05-16,2,1,NaN,横浜市三ツ沢総合公園球技場,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,15454.0
2,11875.0,市原,NaN,1.0,広島,NaN,51,2018-11-08 13:59:00,1993-05-16,2,1,NaN,広島スタジアム,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,33000.0
3,10898.0,名古屋,NaN,1.0,鹿島,NaN,52,2018-11-08 16:00:00,1993-05-16,2,1,NaN,県立カシマサッカースタジアム,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,37496.0
4,19580.0,浦和,NaN,1.0,Ｇ大阪,NaN,53,2018-11-08 19:04:00,1993-05-16,2,1,NaN,万博記念競技場,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,21000.0


# weather

In [288]:
#シンプルに単語を含んでいるか否かで判定(BoWみたい)
for condition in ["晴", "曇", "雨", "屋内", "雷", "雪", "霧", "のち", "時々", "一時"]:
    total_df["weather_{}".format(condition)] = total_df["weather"].str.contains(condition)

In [289]:
total_df.head()

,attendance,away_team,broadcasters,division,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,home_team_win,home_team_score,away_team_score,match_date_year,match_date_month,match_date_day,match_date_dayofweek,date,public_holiday,capacity,weather_晴,weather_曇,weather_雨,weather_屋内,weather_雷,weather_雪,weather_霧,weather_のち,weather_時々,weather_一時
0,59626.0,横浜Ｍ,NaN,1.0,Ｖ川崎,NaN,49,2018-11-08 19:29:00,1993-05-15,1,1,NaN,国立霞ヶ丘競技場,晴,NaN,NaN,NaN,1993,5,15,5,1993-05-15,0,48000.0,True,False,False,False,False,False,False,False,False,False
1,14126.0,清水,NaN,1.0,横浜Ｆ,NaN,50,2018-11-08 13:05:00,1993-05-16,2,1,NaN,横浜市三ツ沢総合公園球技場,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,15454.0,True,False,False,False,False,False,False,False,False,False
2,11875.0,市原,NaN,1.0,広島,NaN,51,2018-11-08 13:59:00,1993-05-16,2,1,NaN,広島スタジアム,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,33000.0,True,False,False,False,False,False,False,False,False,False
3,10898.0,名古屋,NaN,1.0,鹿島,NaN,52,2018-11-08 16:00:00,1993-05-16,2,1,NaN,県立カシマサッカースタジアム,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,37496.0,True,False,False,False,False,False,False,False,False,False
4,19580.0,浦和,NaN,1.0,Ｇ大阪,NaN,53,2018-11-08 19:04:00,1993-05-16,2,1,NaN,万博記念競技場,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,21000.0,True,False,False,False,False,False,False,False,False,False


# RMSLE

In [290]:
total_df["log_attendance"] = np.log1p(total_df["attendance"] + 1)

In [291]:
total_df.head()

,attendance,away_team,broadcasters,division,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,home_team_win,home_team_score,away_team_score,match_date_year,match_date_month,match_date_day,match_date_dayofweek,date,public_holiday,capacity,weather_晴,weather_曇,weather_雨,weather_屋内,weather_雷,weather_雪,weather_霧,weather_のち,weather_時々,weather_一時,log_attendance
0,59626.0,横浜Ｍ,NaN,1.0,Ｖ川崎,NaN,49,2018-11-08 19:29:00,1993-05-15,1,1,NaN,国立霞ヶ丘競技場,晴,NaN,NaN,NaN,1993,5,15,5,1993-05-15,0,48000.0,True,False,False,False,False,False,False,False,False,False,10.995881
1,14126.0,清水,NaN,1.0,横浜Ｆ,NaN,50,2018-11-08 13:05:00,1993-05-16,2,1,NaN,横浜市三ツ沢総合公園球技場,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,15454.0,True,False,False,False,False,False,False,False,False,False,9.555914
2,11875.0,市原,NaN,1.0,広島,NaN,51,2018-11-08 13:59:00,1993-05-16,2,1,NaN,広島スタジアム,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,33000.0,True,False,False,False,False,False,False,False,False,False,9.382359
3,10898.0,名古屋,NaN,1.0,鹿島,NaN,52,2018-11-08 16:00:00,1993-05-16,2,1,NaN,県立カシマサッカースタジアム,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,37496.0,True,False,False,False,False,False,False,False,False,False,9.296518
4,19580.0,浦和,NaN,1.0,Ｇ大阪,NaN,53,2018-11-08 19:04:00,1993-05-16,2,1,NaN,万博記念競技場,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,21000.0,True,False,False,False,False,False,False,False,False,False,9.882366


# team

In [292]:
teams = set(total_df["home_team"].unique().tolist() + total_df["away_team"].unique().tolist())
for team in teams:
    total_df["team_" + team] = (total_df["home_team"] == team) | (total_df["away_team"] == team)

In [293]:
print(total_df.shape)
total_df.head()

(11040, 71)


,attendance,away_team,broadcasters,division,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,home_team_win,home_team_score,away_team_score,match_date_year,match_date_month,match_date_day,match_date_dayofweek,date,public_holiday,capacity,weather_晴,weather_曇,weather_雨,weather_屋内,weather_雷,weather_雪,weather_霧,weather_のち,weather_時々,weather_一時,log_attendance,team_ＦＣ東京,team_Ｃ大阪,team_千葉,team_浦和,team_平塚,team_柏,team_福岡,team_横浜Ｆ,team_甲府,team_横浜ＦＭ,team_京都,team_山形,team_大分,team_広島,team_湘南,team_松本,team_Ｖ川崎,team_名古屋,team_Ｇ大阪,team_横浜ＦＣ,team_徳島,team_神戸,team_鹿島,team_横浜Ｍ,team_清水,team_仙台,team_札幌,team_市原,team_鳥栖,team_川崎Ｆ,team_東京Ｖ,team_長崎,team_Ｆ東京,team_新潟,team_磐田,team_大宮
0,59626.0,横浜Ｍ,NaN,1.0,Ｖ川崎,NaN,49,2018-11-08 19:29:00,1993-05-15,1,1,NaN,国立霞ヶ丘競技場,晴,NaN,NaN,NaN,1993,5,15,5,1993-05-15,0,48000.0,True,False,False,False,False,False,False,False,False,False,10.995881,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
1,14126.0,清水,NaN,1.0,横浜Ｆ,NaN,50,2018-11-08 13:05:00,1993-05-16,2,1,NaN,横浜市三ツ沢総合公園球技場,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,15454.0,True,False,False,False,False,False,False,False,False,False,9.555914,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
2,11875.0,市原,NaN,1.0,広島,NaN,51,2018-11-08 13:59:00,1993-05-16,2,1,NaN,広島スタジアム,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,33000.0,True,False,False,False,False,False,False,False,False,False,9.382359,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3,10898.0,名古屋,NaN,1.0,鹿島,NaN,52,2018-11-08 16:00:00,1993-05-16,2,1,NaN,県立カシマサッカースタジアム,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,37496.0,True,False,False,False,False,False,False,False,False,False,9.296518,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
4,19580.0,浦和,NaN,1.0,Ｇ大阪,NaN,53,2018-11-08 19:04:00,1993-05-16,2,1,NaN,万博記念競技場,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,21000.0,True,False,False,False,False,False,False,False,False,False,9.882366,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


# away_teamとhome_teamをonehot

In [294]:
team_df  = total_df.loc[:,["home_team","away_team"]] 

In [295]:
le = LabelEncoder()
hometeam_label = le.fit_transform(team_df["home_team"])
hometeam_label_df = pd.DataFrame(hometeam_label)
hometeam_label_df.columns = ["hometeam_label"]
awayteam_label = le.transform(team_df["away_team"])
awayteam_label_df = pd.DataFrame(awayteam_label)
awayteam_label_df.columns = ["awayteam_label"]

In [296]:
ohe = OneHotEncoder()
hometeam_ohv_df = pd.DataFrame(ohe.fit_transform(hometeam_label_df).A)
hometeam_ohv_df.columns = ["home"+le.classes_]
print(hometeam_ohv_df.shape)
hometeam_ohv_df.head()

(11040, 36)


,home京都,home仙台,home千葉,home名古屋,home大分,home大宮,home山形,home川崎Ｆ,home市原,home平塚,home広島,home徳島,home新潟,home札幌,home東京Ｖ,home松本,home柏,home横浜Ｆ,home横浜ＦＣ,home横浜ＦＭ,home横浜Ｍ,home浦和,home清水,home湘南,home甲府,home磐田,home神戸,home福岡,home長崎,home鳥栖,home鹿島,homeＣ大阪,homeＦ東京,homeＦＣ東京,homeＧ大阪,homeＶ川崎
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [297]:
awayteam_ohv_df = pd.DataFrame(ohe.transform(awayteam_label_df).A)
awayteam_ohv_df.columns = ["away"+le.classes_]
print(awayteam_ohv_df.shape)
awayteam_ohv_df.head()

(11040, 36)


,away京都,away仙台,away千葉,away名古屋,away大分,away大宮,away山形,away川崎Ｆ,away市原,away平塚,away広島,away徳島,away新潟,away札幌,away東京Ｖ,away松本,away柏,away横浜Ｆ,away横浜ＦＣ,away横浜ＦＭ,away横浜Ｍ,away浦和,away清水,away湘南,away甲府,away磐田,away神戸,away福岡,away長崎,away鳥栖,away鹿島,awayＣ大阪,awayＦ東京,awayＦＣ東京,awayＧ大阪,awayＶ川崎
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [298]:
total_df = pd.concat((total_df, awayteam_ohv_df), axis=1)
total_df = pd.concat((total_df, hometeam_ohv_df), axis=1)
print(total_df.shape)
total_df.head()

(11040, 143)


,attendance,away_team,broadcasters,division,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,home_team_win,home_team_score,away_team_score,match_date_year,match_date_month,match_date_day,match_date_dayofweek,date,public_holiday,capacity,weather_晴,weather_曇,weather_雨,weather_屋内,weather_雷,weather_雪,weather_霧,weather_のち,weather_時々,weather_一時,log_attendance,team_ＦＣ東京,team_Ｃ大阪,team_千葉,team_浦和,team_平塚,team_柏,team_福岡,team_横浜Ｆ,team_甲府,team_横浜ＦＭ,team_京都,team_山形,team_大分,team_広島,team_湘南,team_松本,team_Ｖ川崎,team_名古屋,team_Ｇ大阪,team_横浜ＦＣ,team_徳島,team_神戸,team_鹿島,team_横浜Ｍ,team_清水,team_仙台,team_札幌,team_市原,team_鳥栖,team_川崎Ｆ,team_東京Ｖ,team_長崎,team_Ｆ東京,team_新潟,team_磐田,team_大宮,away京都,away仙台,away千葉,away名古屋,away大分,away大宮,away山形,away川崎Ｆ,away市原,away平塚,away広島,away徳島,away新潟,away札幌,away東京Ｖ,away松本,away柏,away横浜Ｆ,away横浜ＦＣ,away横浜ＦＭ,away横浜Ｍ,away浦和,away清水,away湘南,away甲府,away磐田,away神戸,away福岡,away長崎,away鳥栖,away鹿島,awayＣ大阪,awayＦ東京,awayＦＣ東京,awayＧ大阪,awayＶ川崎,home京都,home仙台,home千葉,home名古屋,home大分,home大宮,home山形,home川崎Ｆ,home市原,home平塚,home広島,home徳島,home新潟,home札幌,home東京Ｖ,home松本,home柏,home横浜Ｆ,home横浜ＦＣ,home横浜ＦＭ,home横浜Ｍ,home浦和,home清水,home湘南,home甲府,home磐田,home神戸,home福岡,home長崎,home鳥栖,home鹿島,homeＣ大阪,homeＦ東京,homeＦＣ東京,homeＧ大阪,homeＶ川崎
0,59626.0,横浜Ｍ,NaN,1.0,Ｖ川崎,NaN,49,2018-11-08 19:29:00,1993-05-15,1,1,NaN,国立霞ヶ丘競技場,晴,NaN,NaN,NaN,1993,5,15,5,1993-05-15,0,48000.0,True,False,False,False,False,False,False,False,False,False,10.995881,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,14126.0,清水,NaN,1.0,横浜Ｆ,NaN,50,2018-11-08 13:05:00,1993-05-16,2,1,NaN,横浜市三ツ沢総合公園球技場,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,15454.0,True,False,False,False,False,False,False,False,False,False,9.555914,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11875.0,市原,NaN,1.0,広島,NaN,51,2018-11-08 13:59:00,1993-05-16,2,1,NaN,広島スタジアム,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,33000.0,True,False,False,False,False,False,False,False,False,False,9.382359,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10898.0,名古屋,NaN,1.0,鹿島,NaN,52,2018-11-08 16:00:00,1993-05-16,2,1,NaN,県立カシマサッカースタジアム,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,37496.0,True,False,False,False,False,False,False,False,False,False,9.296518,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

# section,match_date_month,match_date_day,match_dayofweekもohv

In [299]:
columns = ["section","match_date_month","match_date_day","match_date_dayofweek"]
for column in columns:
    le = LabelEncoder()
    label = le.fit_transform(total_df[column])
    label_df = pd.DataFrame(label)
    label_df.columns = [column + "_label"]
    ohe = OneHotEncoder()
    ohv_df = pd.DataFrame(ohe.fit_transform(label_df).A)
    ohv_df.columns = np.array(column,dtype=object)+le.classes_.astype(str)
    total_df = pd.concat((total_df,ohv_df),axis=1)
    print(total_df.shape)


(11040, 177)
(11040, 188)
(11040, 219)
(11040, 226)


In [300]:
total_df.head()

,attendance,away_team,broadcasters,division,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,home_team_win,home_team_score,away_team_score,match_date_year,match_date_month,match_date_day,match_date_dayofweek,date,public_holiday,capacity,weather_晴,weather_曇,weather_雨,weather_屋内,weather_雷,weather_雪,weather_霧,weather_のち,weather_時々,weather_一時,log_attendance,team_ＦＣ東京,team_Ｃ大阪,team_千葉,team_浦和,team_平塚,team_柏,team_福岡,team_横浜Ｆ,team_甲府,team_横浜ＦＭ,team_京都,team_山形,team_大分,team_広島,team_湘南,team_松本,team_Ｖ川崎,team_名古屋,team_Ｇ大阪,team_横浜ＦＣ,team_徳島,team_神戸,team_鹿島,team_横浜Ｍ,team_清水,team_仙台,team_札幌,team_市原,team_鳥栖,team_川崎Ｆ,team_東京Ｖ,team_長崎,team_Ｆ東京,team_新潟,team_磐田,team_大宮,away京都,away仙台,away千葉,away名古屋,away大分,away大宮,away山形,away川崎Ｆ,away市原,away平塚,away広島,away徳島,away新潟,away札幌,away東京Ｖ,away松本,away柏,away横浜Ｆ,away横浜ＦＣ,away横浜ＦＭ,away横浜Ｍ,away浦和,away清水,away湘南,away甲府,away磐田,away神戸,away福岡,away長崎,away鳥栖,away鹿島,awayＣ大阪,awayＦ東京,awayＦＣ東京,awayＧ大阪,awayＶ川崎,home京都,home仙台,home千葉,home名古屋,home大分,home大宮,home山形,home川崎Ｆ,home市原,home平塚,home広島,home徳島,home新潟,home札幌,home東京Ｖ,home松本,home柏,home横浜Ｆ,home横浜ＦＣ,home横浜ＦＭ,home横浜Ｍ,home浦和,home清水,home湘南,home甲府,home磐田,home神戸,home福岡,home長崎,home鳥栖,home鹿島,homeＣ大阪,homeＦ東京,homeＦＣ東京,homeＧ大阪,homeＶ川崎,section1,section2,section3,section4,section5,section6,section7,section8,section9,section10,section11,section12,section13,section14,section15,section16,section17,section18,section19,section20,section21,section22,section23,section24,section25,section26,section27,section28,section29,section30,section31,section32,section33,section34,match_date_month2,match_date_month3,match_date_month4,match_date_month5,match_date_month6,match_date_month7,match_date_month8,match_date_month9,match_date_month10,match_date_month11,match_date_month12,match_date_day1,match_date_day2,match_date_day3,match_date_day4,match_date_day5,match_date_day6,match_date_day7,match_date_day8,match_date_day9,match_date_day10,match_date_day11,match_date_day12,match_date_day13,match_date_day14,match_date_day15,match_date_day16,match_date_day17,match_date_day18,match_date_day19,match_date_day20,match_date_day21,match_date_day22,match_date_day23,match_date_day24,match_date_day25,match_date_day26,match_date_day27,match_date_day28,match_date_day29,match_date_day30,match_date_day31,match_date_dayofweek0,match_date_dayofweek1,match_date_dayofweek2,match_date_dayofweek3,match_date_dayofweek4,match_date_dayofweek5,match_date_dayofweek6
0,59626.0,横浜Ｍ,NaN,1.0,Ｖ川崎,NaN,49,2018-11-08 19:29:00,1993-05-15,1,1,NaN,国立霞ヶ丘競技場,晴,NaN,NaN,NaN,1993,5,15,5,1993-05-15,0,48000.0,True,False,False,False,False,False,False,False,False,False,10.995881,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,14126.0,清水,NaN,1.0,横浜Ｆ,NaN,50,2018-11-08 13:05:00,1993-05-16,2,1,NaN,横浜市三ツ沢総合公園球技場,晴,NaN,NaN,NaN,1993,5,16,6,1993-05-16,0,15454.0,True,False,False,False,False,False,False,False,False,False,9.555914,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [301]:
column = "section"
le = LabelEncoder()
label = le.fit_transform(total_df[column])
label_df = pd.DataFrame(label)
label_df.columns = [column + "_label"]
label_df.head()

,section_label
0,0
1,0
2,0
3,0
4,0


In [302]:
np.array("home",dtype=object)+le.classes_.astype(str)

array(['home1', 'home2', 'home3', 'home4', 'home5', 'home6', 'home7',
       'home8', 'home9', 'home10', 'home11', 'home12', 'home13', 'home14',
       'home15', 'home16', 'home17', 'home18', 'home19', 'home20',
       'home21', 'home22', 'home23', 'home24', 'home25', 'home26',
       'home27', 'home28', 'home29', 'home30', 'home31', 'home32',
       'home33', 'home34'], dtype=object)

In [303]:
ohe = OneHotEncoder()
ohv_df = pd.DataFrame(ohe.fit_transform(label_df).A)
ohv_df.columns = np.array(column,dtype=object)+le.classes_.astype(str)
print(ohv_df.shape)
ohv_df.head()

(11040, 34)


,section1,section2,section3,section4,section5,section6,section7,section8,section9,section10,section11,section12,section13,section14,section15,section16,section17,section18,section19,section20,section21,section22,section23,section24,section25,section26,section27,section28,section29,section30,section31,section32,section33,section34
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [304]:
total_df = pd.concat((total_df,ohv_df),axis=1)
print(total_df.shape)
total_df.head()

(11040, 260)


,attendance,away_team,broadcasters,division,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,home_team_win,home_team_score,away_team_score,match_date_year,match_date_month,match_date_day,match_date_dayofweek,date,public_holiday,capacity,weather_晴,weather_曇,weather_雨,weather_屋内,weather_雷,weather_雪,weather_霧,weather_のち,weather_時々,weather_一時,log_attendance,team_ＦＣ東京,team_Ｃ大阪,team_千葉,team_浦和,team_平塚,team_柏,team_福岡,team_横浜Ｆ,team_甲府,team_横浜ＦＭ,team_京都,team_山形,team_大分,team_広島,team_湘南,team_松本,team_Ｖ川崎,team_名古屋,team_Ｇ大阪,team_横浜ＦＣ,team_徳島,team_神戸,team_鹿島,team_横浜Ｍ,team_清水,team_仙台,team_札幌,team_市原,team_鳥栖,team_川崎Ｆ,team_東京Ｖ,team_長崎,team_Ｆ東京,team_新潟,team_磐田,team_大宮,away京都,away仙台,away千葉,away名古屋,away大分,away大宮,away山形,away川崎Ｆ,away市原,away平塚,away広島,away徳島,away新潟,away札幌,away東京Ｖ,away松本,away柏,away横浜Ｆ,away横浜ＦＣ,away横浜ＦＭ,away横浜Ｍ,away浦和,away清水,away湘南,away甲府,away磐田,away神戸,away福岡,away長崎,away鳥栖,away鹿島,awayＣ大阪,awayＦ東京,awayＦＣ東京,awayＧ大阪,awayＶ川崎,home京都,home仙台,home千葉,home名古屋,home大分,home大宮,home山形,home川崎Ｆ,home市原,home平塚,home広島,home徳島,home新潟,home札幌,home東京Ｖ,home松本,home柏,home横浜Ｆ,home横浜ＦＣ,home横浜ＦＭ,home横浜Ｍ,home浦和,home清水,home湘南,home甲府,home磐田,home神戸,home福岡,home長崎,home鳥栖,home鹿島,homeＣ大阪,homeＦ東京,homeＦＣ東京,homeＧ大阪,homeＶ川崎,section1,section2,section3,section4,section5,section6,section7,section8,section9,section10,section11,section12,section13,section14,section15,section16,section17,section18,section19,section20,section21,section22,section23,section24,section25,section26,section27,section28,section29,section30,section31,section32,section33,section34,match_date_month2,match_date_month3,match_date_month4,match_date_month5,match_date_month6,match_date_month7,match_date_month8,match_date_month9,match_date_month10,match_date_month11,match_date_month12,match_date_day1,match_date_day2,match_date_day3,match_date_day4,match_date_day5,match_date_day6,match_date_day7,match_date_day8,match_date_day9,match_date_day10,match_date_day11,match_date_day12,match_date_day13,match_date_day14,match_date_day15,match_date_day16,match_date_day17,match_date_day18,match_date_day19,match_date_day20,match_date_day21,match_date_day22,match_date_day23,match_date_day24,match_date_day25,match_date_day26,match_date_day27,match_date_day28,match_date_day29,match_date_day30,match_date_day31,match_date_dayofweek0,match_date_dayofweek1,match_date_dayofweek2,match_date_dayofweek3,match_date_dayofweek4,match_date_dayofweek5,match_date_dayofweek6,section1,section2,section3,section4,section5,section6,section7,section8,section9,section10,section11,section12,section13,section14,section15,section16,section17,section18,section19,section20,section21,section22,section23,section24,section25,section26,section27,section28,section29,section30,section31,section32,section33,section34
0,59626.0,横浜Ｍ,NaN,1.0,Ｖ川崎,NaN,49,2018-11-08 19:29:00,1993-05-15,1,1,NaN,国立霞ヶ丘競技場,晴,NaN,NaN,NaN,1993,5,15,5,1993-05-15,0,48000.0,True,False,False,False,False,False,False,False,False,False,10.995881,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14126.0,清水,NaN,1.0,横浜Ｆ,NaN,50,2018-11-08 13:05:00,1993-05-16,2,1,NaN

# スタジアムもohv

In [305]:
column = "venue"
le = LabelEncoder()
label = le.fit_transform(total_df[column])
label_df = pd.DataFrame(label)
label_df.columns = [column + "_label"]
ohe = OneHotEncoder()
ohv_df = pd.DataFrame(ohe.fit_transform(label_df).A)
ohv_df.columns = np.array(column,dtype=object)+le.classes_.astype(str)
total_df = pd.concat((total_df,ohv_df),axis=1)
print(total_df.shape)


(11040, 375)


In [306]:
total_df.head()

,attendance,away_team,broadcasters,division,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,home_team_win,home_team_score,away_team_score,match_date_year,match_date_month,match_date_day,match_date_dayofweek,date,public_holiday,capacity,weather_晴,weather_曇,weather_雨,weather_屋内,weather_雷,weather_雪,weather_霧,weather_のち,weather_時々,weather_一時,log_attendance,team_ＦＣ東京,team_Ｃ大阪,team_千葉,team_浦和,team_平塚,team_柏,team_福岡,team_横浜Ｆ,team_甲府,team_横浜ＦＭ,team_京都,team_山形,team_大分,team_広島,team_湘南,team_松本,team_Ｖ川崎,team_名古屋,team_Ｇ大阪,team_横浜ＦＣ,team_徳島,team_神戸,team_鹿島,team_横浜Ｍ,team_清水,team_仙台,team_札幌,team_市原,team_鳥栖,team_川崎Ｆ,team_東京Ｖ,team_長崎,team_Ｆ東京,team_新潟,team_磐田,team_大宮,away京都,away仙台,away千葉,away名古屋,away大分,away大宮,away山形,away川崎Ｆ,away市原,away平塚,away広島,away徳島,away新潟,away札幌,away東京Ｖ,away松本,away柏,away横浜Ｆ,away横浜ＦＣ,away横浜ＦＭ,away横浜Ｍ,away浦和,away清水,away湘南,away甲府,away磐田,away神戸,away福岡,away長崎,away鳥栖,away鹿島,awayＣ大阪,awayＦ東京,awayＦＣ東京,awayＧ大阪,awayＶ川崎,home京都,home仙台,home千葉,home名古屋,home大分,home大宮,home山形,home川崎Ｆ,home市原,home平塚,home広島,home徳島,home新潟,home札幌,home東京Ｖ,home松本,home柏,home横浜Ｆ,home横浜ＦＣ,home横浜ＦＭ,home横浜Ｍ,home浦和,home清水,home湘南,home甲府,home磐田,home神戸,home福岡,home長崎,home鳥栖,home鹿島,homeＣ大阪,homeＦ東京,homeＦＣ東京,homeＧ大阪,homeＶ川崎,section1,section2,section3,section4,section5,section6,section7,section8,section9,section10,section11,section12,section13,section14,section15,section16,section17,section18,section19,section20,section21,section22,section23,section24,section25,section26,section27,section28,section29,section30,section31,section32,section33,section34,match_date_month2,match_date_month3,match_date_month4,match_date_month5,match_date_month6,match_date_month7,match_date_month8,match_date_month9,match_date_month10,match_date_month11,match_date_month12,match_date_day1,match_date_day2,match_date_day3,match_date_day4,match_date_day5,match_date_day6,match_date_day7,match_date_day8,match_date_day9,match_date_day10,match_date_day11,match_date_day12,match_date_day13,match_date_day14,match_date_day15,match_date_day16,match_date_day17,match_date_day18,match_date_day19,match_date_day20,match_date_day21,match_date_day22,match_date_day23,match_date_day24,match_date_day25,match_date_day26,match_date_day27,match_date_day28,match_date_day29,match_date_day30,match_date_day31,match_date_dayofweek0,match_date_dayofweek1,match_date_dayofweek2,match_date_dayofweek3,match_date_dayofweek4,match_date_dayofweek5,match_date_dayofweek6,section1,section2,section3,section4,section5,section6,section7,section8,section9,section10,section11,section12,section13,section14,section15,section16,section17,section18,section19,section20,section21,section22,section23,section24,section25,section26,section27,section28,section29,section30,section31,section32,section33,section34,venueさいたま市大宮公園サッカー場,venueさいたま市浦和駒場スタジアム,venueびんご運動公園陸上競技場,venueアウトソーシングスタジアム日本平,venueエコパスタジアム,venueエディオンスタジアム広島,venueキンチョウスタジアム,venueコカ・コーラウエスト広島スタジアム,venueジュビロ磐田サッカースタジアム,venueジュビロ磐田スタジアム,venueテクノポート福井スタジアム,venueデンカビッグスワンスタジアム,venueトランスコスモススタジアム長崎,venueニッパツ三ツ沢球技場,venueノエビアスタジアム神戸,venueパナソニックスタジアム吹田,venueパロマ瑞穂スタジアム,venueフクダ電子アリーナ,venueベストアメニティスタジアム,venueホームズスタジアム神戸,venueヤマハスタジアム（磐田）,venueヤンマースタジアム長居,venueユアテックスタジアム仙台,venueレベルファイブスタジアム,venue七北田公園仙台スタジアム,venue万博記念競技場,venue三協フロンテア柏スタジアム,venue三重県営鈴鹿スポーツガーデン,venue九州石油ドーム,venue京都市西京極総合運動公園陸上競技場,venue京都市西京極総合運動公園陸上競技場兼球技場,venue仙台スタジアム,venue佐賀県総合運動場陸上競技場,venue函館市千代台公園陸上競技場,venue千代台公園陸上競技場,venue博多の森陸上競技場,venue名古屋市港サッカー場,venue名古屋市瑞穂球技場,venue名古屋市瑞穂陸上競技場,venue味の素スタジアム,venue国立競技場,venue国立霞ヶ丘競技場,venue埼玉スタジアム２００２,venue埼玉県営大宮公園サッカー場,venue大分スポーツ公園総合競技場,venue大分市営陸上競技場,venue大分銀行ドーム,venue大阪長居スタジアム,venue大阪長居第２陸上競技場,venue室蘭市入江運動公園陸上競技場,venue宮城スタジアム,venue宮城陸上競技場,venue富山県総合運動公園陸上競技場,venue山形県総合運動公園陸上競技場,venue山梨中銀スタジアム,venue山梨県小瀬スポーツ公園陸上競技場,venue岐阜メモリアルセンター長良川競技場,venue岡山県陸上競技場桃太郎スタジアム,venue岩手県営陸上競技場,venue市原臨海競技場,venue市立吹田サッカースタジアム,venue平塚競技場,venue広島スタジアム,venue広島ビッグアーチ,venue徳島県鳴門総合運動公園陸上競技場,venue愛媛県総合運動公園陸上競技場,venue新潟スタジアム,venue新潟市陸上競技場,venue日本平スタジアム,venue日本平運動公園球技場,venue日産スタジアム,venue日立柏サッカー場,ve

# 保存

In [307]:
print(total_df.shape)
total_df.to_csv("../data/ML_feature.csv", index=False)

(11040, 375)
